In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert import download
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import os
from transformers import BertModel
from zipfile import ZipFile

In [147]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

In [36]:
def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }
    
    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)

In [37]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_v1.zip
using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [119]:
df = pd.read_csv(r'C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\input_data.csv',index_col=0)
df

,제목,상승하락
0,"대한생명, 원금 100% 환급해주는 변액보험 출시",0
1,"코스피, 새해 첫날 혼조세",0
2,[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다,0
3,"<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염",0
4,"[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.",0
5,“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>,0
6,[투자전략]조정 우려 주식비중 줄여야-김병연,0
7,"[초점]""1월 효과 있다 1750까지 가능""",0
8,"코스피, 1690 탈환..1691.10",0
9,[초점]'1월 효과' 있다 1750까지 가능,0


In [112]:
encoder = LabelEncoder()
encoder.fit(df['상승하락'])
df['상승하락'] = encoder.transform(df['상승하락'])
df

,제목,상승하락
0,"대한생명, 원금 100% 환급해주는 변액보험 출시",0
1,"코스피, 새해 첫날 혼조세",0
2,[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다,0
3,"<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염",0
4,"[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.",0
5,“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>,0
6,[투자전략]조정 우려 주식비중 줄여야-김병연,0
7,"[초점]""1월 효과 있다 1750까지 가능""",0
8,"코스피, 1690 탈환..1691.10",0
9,[초점]'1월 효과' 있다 1750까지 가능,0


In [126]:
df['제목'] = df['제목'].astype(str)
df['상승하락'] = df['상승하락'].astype(str)

In [127]:
data = df.values.tolist()
data

[['대한생명, 원금 100% 환급해주는 변액보험 출시', '0'],
 ['코스피, 새해 첫날 혼조세', '0'],
 ['[2010 경제대전망]짝수해 징크스는 없다..코스피 2120찍는다', '0'],
 ['<생생코스닥>주간추천주 결산-한화證, 575% 수익률 기염', '0'],
 ['[타이쿤의 시장분석] 전자주, 2차 상승 국면이다.', '0'],
 ['“올 증시 20%수익 저평가 종목 공략”<펀드매니저 설문>', '0'],
 ['[투자전략]조정 우려 주식비중 줄여야-김병연', '0'],
 ['[초점]"1월 효과 있다 1750까지 가능"', '0'],
 ['코스피, 1690 탈환..1691.10', '0'],
 ["[초점]'1월 효과' 있다 1750까지 가능", '0'],
 ["NH-CA운용 '레버리지 인덱스펀드', 금감원 선정 금융신상품 최우수상", '0'],
 ['작년 파생상품 거래 사상 최고', '0'],
 ['반도체주 “해피 뉴 이어”', '0'],
 ['코스피, 오름폭 확대 1690선 탈환', '0'],
 ['추가 상승이냐 vs 게걸음 장세냐', '0'],
 ['[생생코스닥]우리투자ㆍ대신證 ‘탁월한 안목’ 빛났다', '0'],
 ["대주주가 가장 맛있어 하는 '유상감자'", '0'],
 ["코스피 소폭 상승 외국인 기관 '사자'", '0'],
 ['올 첫 증시 큰폭 상승 虎 虎 虎 호쾌한 스타트', '0'],
 ['코스피, 경인년 기대감 매수세 유입 - 동시호가', '0'],
 ['한화證, 코스피200 스텝다운 ELS 등 2종 판매', '0'],
 ['LED 2차전지 등 녹색업종 올해도 ‘강추’', '0'],
 ['"올 증시 上低下高 1,900까지 오를것"', '0'],
 ["2010년 '白호랑이 해' 증시 개막", '0'],
 ['코스닥 520선 재돌파 코스피는 강보합', '0'],
 ['[증권가 소식] 한국투자증권 外', '0'],
 ["코스피 1690선 돌파 외국인 기관 '사자'", '0'],
 ['한화證, 코스피200스텝다운 ELS 2종 판매

In [128]:
train, test = train_test_split(data, test_size = 0.2, shuffle = False, random_state=777)
print(len(train))
print(len(test))

173617
43405


In [129]:
tokenizer = get_tokenizer()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [130]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [131]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [132]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [140]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [158]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size) # num_workers=5
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size) #num_workers=5

In [159]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [160]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [161]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.7652342915534973 train acc 0.421875
epoch 1 batch id 201 loss 0.6739198565483093 train acc 0.4847636815920398
epoch 1 batch id 401 loss 1.2557337284088135 train acc 0.5063902743142145
epoch 1 batch id 601 loss 0.8614548444747925 train acc 0.4865848585690516
